In [1]:
from google.colab import drive
drive.mount('/content/drive')

!pip install opencv-python mediapipe torch torchvision torchaudio --quiet
!pip install pandas scikit-learn --quiet


Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 12.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.71.2 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 4.25.8 which is incompatible.
opentelemetry-proto 1.37.0 requires protobuf<7.0,>=5.0, but you have protobuf 4.25.8 which is incompatible.
ydf 0.13.0 requires protobuf<7.0.0,>=5.29.1, but you have protobuf 4.25.8 which is incompatible.


In [2]:
import zipfile
import os

zip_path = "/content/drive/MyDrive/LSC/DataSet_LSC-main.zip"
extract_dir = "/content/DataSet_LSC-main"

os.makedirs(extract_dir, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as z:
    z.extractall(extract_dir)

print("Contenido de la carpeta raíz del dataset:")
print(os.listdir(extract_dir))


Contenido de la carpeta raíz del dataset:
['DataSet_LSC-main']


In [3]:
import os

root_dir = "/content/DataSet_LSC-main/DataSet_LSC-main"

print("Carpetas de glosas:")
print(os.listdir(root_dir)[:30])  # primeras 30 para ver


Carpetas de glosas:
['buenos-dias', 'hola', 'reunion', 'pais', 'tarde', 'llegar', 'el-ella', 'otro', 'mujer', 'jugar', 'pasado', 'balon', 'adulto', 'dormir', 'algunos', 'año', 'un', 'sordo', 'necesitar', 'anciano', 'yo', 'futbol', 'hijo', 'bus', 'casas', 'ir', 'comprar', 'ellos-dos-ellas-dos', 'buenas-noches', 'ya']


In [4]:
import pandas as pd

rows = []
for gloss in sorted(os.listdir(root_dir)):
    gloss_dir = os.path.join(root_dir, gloss)
    if not os.path.isdir(gloss_dir):
        continue
    for fname in os.listdir(gloss_dir):
        if fname.lower().endswith((".mp4", ".mov", ".mkv", ".avi")):
            video_path = os.path.join(gloss_dir, fname)
            rows.append({
                "video_path": video_path,
                "gloss": gloss
            })

df = pd.DataFrame(rows)
print(df.head())
print("Total de videos:", len(df))

csv_path = "/content/annotations_LSC.csv"
df.to_csv(csv_path, index=False)
csv_path


                                          video_path   gloss
0  /content/DataSet_LSC-main/DataSet_LSC-main/adu...  adulto
1  /content/DataSet_LSC-main/DataSet_LSC-main/adu...  adulto
2  /content/DataSet_LSC-main/DataSet_LSC-main/adu...  adulto
3  /content/DataSet_LSC-main/DataSet_LSC-main/adu...  adulto
4  /content/DataSet_LSC-main/DataSet_LSC-main/adu...  adulto
Total de videos: 526


'/content/annotations_LSC.csv'

In [5]:
unique_glosses = sorted(df["gloss"].unique())
print("Total de glosas:", len(unique_glosses))
print(unique_glosses)

# Usaremos solo 30 glosas para que el entrenamiento sea manejable
subset_glosses = unique_glosses[:30]
print("Glosas que usaremos:", subset_glosses)

df = df[df["gloss"].isin(subset_glosses)].reset_index(drop=True)
print("Videos luego de filtrar:", len(df))
df.head()


Total de glosas: 41
['adulto', 'algunos', 'anciano', 'año', 'balon', 'bebe', 'buenas-noches', 'buenos-dias', 'bus', 'cada-uno', 'casas', 'comprar', 'dormir', 'el-ella', 'ellas-ellos', 'ellos-dos-ellas-dos', 'futbol', 'futuro', 'hambre', 'hijo', 'hola', 'hombre', 'ir', 'jugar', 'llegar', 'mujer', 'nacer', 'necesitar', 'otro', 'pais', 'pasado', 'pedalear-bicleta', 'persona', 'reunion', 'sordo', 'tarde', 'un', 'ver', 'visitar', 'ya', 'yo']
Glosas que usaremos: ['adulto', 'algunos', 'anciano', 'año', 'balon', 'bebe', 'buenas-noches', 'buenos-dias', 'bus', 'cada-uno', 'casas', 'comprar', 'dormir', 'el-ella', 'ellas-ellos', 'ellos-dos-ellas-dos', 'futbol', 'futuro', 'hambre', 'hijo', 'hola', 'hombre', 'ir', 'jugar', 'llegar', 'mujer', 'nacer', 'necesitar', 'otro', 'pais']
Videos luego de filtrar: 380


,video_path,gloss
0,/content/DataSet_LSC-main/DataSet_LSC-main/adu...,adulto
1,/content/DataSet_LSC-main/DataSet_LSC-main/adu...,adulto
2,/content/DataSet_LSC-main/DataSet_LSC-main/adu...,adulto
3,/content/DataSet_LSC-main/DataSet_LSC-main/adu...,adulto
4,/content/DataSet_LSC-main/DataSet_LSC-main/adu...,adulto


In [6]:
from sklearn.model_selection import train_test_split

train_df, temp_df = train_test_split(
    df,
    test_size=0.3,
    stratify=df["gloss"],
    random_state=42
)

val_df, test_df = train_test_split(
    temp_df,
    test_size=0.5,
    stratify=temp_df["gloss"],
    random_state=42
)

print("Train:", len(train_df), "Val:", len(val_df), "Test:", len(test_df))

train_df.to_csv("/content/train_LSC.csv", index=False)
val_df.to_csv("/content/val_LSC.csv", index=False)
test_df.to_csv("/content/test_LSC.csv", index=False)


Train: 266 Val: 57 Test: 57


In [7]:
label2idx = {gloss: i for i, gloss in enumerate(sorted(df["gloss"].unique()))}
idx2label = {i: g for g, i in label2idx.items()}

print("Número de clases:", len(label2idx))
list(label2idx.items())[:10]


Número de clases: 30


[('adulto', 0),
 ('algunos', 1),
 ('anciano', 2),
 ('año', 3),
 ('balon', 4),
 ('bebe', 5),
 ('buenas-noches', 6),
 ('buenos-dias', 7),
 ('bus', 8),
 ('cada-uno', 9)]

In [8]:
import cv2
import numpy as np
import mediapipe as mp

mp_holistic = mp.solutions.holistic
holistic = mp_holistic.Holistic(static_image_mode=False)

def extract_keypoints_from_frame(image):
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = holistic.process(image_rgb)

    keypoints = []

    # Pose (33 puntos)
    if results.pose_landmarks:
        for lm in results.pose_landmarks.landmark:
            keypoints.extend([lm.x, lm.y])
    else:
        keypoints.extend([0.0, 0.0] * 33)

    # Mano izquierda (21 puntos)
    if results.left_hand_landmarks:
        for lm in results.left_hand_landmarks.landmark:
            keypoints.extend([lm.x, lm.y])
    else:
        keypoints.extend([0.0, 0.0] * 21)

    # Mano derecha (21 puntos)
    if results.right_hand_landmarks:
        for lm in results.right_hand_landmarks.landmark:
            keypoints.extend([lm.x, lm.y])
    else:
        keypoints.extend([0.0, 0.0] * 21)

    return np.array(keypoints, dtype=np.float32)


def extract_sequence_from_video(video_path, num_frames=16):
    cap = cv2.VideoCapture(video_path)

    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    if frame_count == 0:
        cap.release()
        return np.zeros((num_frames, (33+21+21)*2), dtype=np.float32)

    indices = np.linspace(0, frame_count - 1, num_frames).astype(int)

    seq = []
    for idx in indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ok, frame = cap.read()
        if not ok:
            seq.append(np.zeros(((33+21+21)*2,), dtype=np.float32))
            continue
        kps = extract_keypoints_from_frame(frame)
        seq.append(kps)

    cap.release()
    return np.stack(seq)


In [9]:
def build_dataset(split_df, num_frames=16, max_videos_per_gloss=None):
    X_list = []
    y_list = []

    # opcional: limitar videos por glosa para que sea más rápido
    if max_videos_per_gloss is not None:
        split_df = split_df.groupby("gloss").head(max_videos_per_gloss).reset_index(drop=True)

    for idx, row in split_df.iterrows():
        seq = extract_sequence_from_video(row["video_path"], num_frames=num_frames)
        X_list.append(seq)
        y_list.append(label2idx[row["gloss"]])

        if (idx + 1) % 20 == 0:
            print(f"Procesados {idx+1} videos...")

    X = np.stack(X_list)   # [N, T, D]
    y = np.array(y_list, dtype=np.int64)
    return X, y

X_train, y_train = build_dataset(train_df, num_frames=16, max_videos_per_gloss=40)
X_val,   y_val   = build_dataset(val_df,   num_frames=16, max_videos_per_gloss=20)
X_test,  y_test  = build_dataset(test_df,  num_frames=16, max_videos_per_gloss=20)

X_train.shape, y_train.shape


/usr/local/lib/python3.12/dist-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Procesados 20 videos...
Procesados 40 videos...
Procesados 60 videos...
Procesados 80 videos...
Procesados 100 videos...
Procesados 120 videos...
Procesados 140 videos...
Procesados 160 videos...
Procesados 180 videos...
Procesados 200 videos...
Procesados 220 videos...
Procesados 240 videos...
Procesados 260 videos...


/usr/local/lib/python3.12/dist-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Procesados 20 videos...
Procesados 40 videos...


/usr/local/lib/python3.12/dist-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Procesados 20 videos...
Procesados 40 videos...


((266, 16, 150), (266,))

In [11]:
np.save("/content/X_train.npy", X_train)
np.save("/content/y_train.npy", y_train)
np.save("/content/X_val.npy", X_val)
np.save("/content/y_val.npy", y_val)
np.save("/content/X_test.npy", X_test)
np.save("/content/y_test.npy", y_test)

out_dir = "/content/drive/MyDrive/LSC/npy"
os.makedirs(out_dir, exist_ok=True)

for name in ["X_train", "y_train", "X_val", "y_val", "X_test", "y_test"]:
    src = f"/content/{name}.npy"
    dst = f"{out_dir}/{name}.npy"
    !cp "$src" "$dst"

print("Guardado todo en:", out_dir)


Guardado todo en: /content/drive/MyDrive/LSC/npy


In [12]:
import numpy as np
import torch

X_train = np.load("/content/X_train.npy")
y_train = np.load("/content/y_train.npy")
X_val   = np.load("/content/X_val.npy")
y_val   = np.load("/content/y_val.npy")

print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)

num_classes = len(label2idx)
input_size  = X_train.shape[2]   # D
seq_len     = X_train.shape[1]   # T

num_classes, input_size, seq_len


(266, 16, 150) (266,)
(57, 16, 150) (57,)


(30, 150, 16)

In [13]:
from torch.utils.data import Dataset, DataLoader

class SignsDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.from_numpy(X).float()
        self.y = torch.from_numpy(y).long()

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_dataset = SignsDataset(X_train, y_train)
val_dataset   = SignsDataset(X_val,   y_val)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader   = DataLoader(val_dataset,   batch_size=16, shuffle=False)


In [14]:
import torch.nn as nn

class SignRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes, num_layers=1):
        super().__init__()
        self.rnn = nn.GRU(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True
        )
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out, h_n = self.rnn(x)      # out: [B, T, H]
        last = out[:, -1, :]        # último paso temporal
        logits = self.fc(last)      # [B, C]
        return logits

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SignRNN(input_size=input_size, hidden_size=128, num_classes=num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)


In [15]:
def train_one_epoch(model, loader, optimizer, criterion, device):
    model.train()
    total_loss = 0.0
    correct = 0
    total = 0

    for X_batch, y_batch in loader:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)

        optimizer.zero_grad()
        logits = model(X_batch)
        loss = criterion(logits, y_batch)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * X_batch.size(0)
        preds = logits.argmax(dim=1)
        correct += (preds == y_batch).sum().item()
        total += X_batch.size(0)

    return total_loss / total, correct / total


def eval_epoch(model, loader, criterion, device):
    model.eval()
    total_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for X_batch, y_batch in loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)

            logits = model(X_batch)
            loss = criterion(logits, y_batch)

            total_loss += loss.item() * X_batch.size(0)
            preds = logits.argmax(dim=1)
            correct += (preds == y_batch).sum().item()
            total += X_batch.size(0)

    return total_loss / total, correct / total


num_epochs = 10
for epoch in range(1, num_epochs + 1):
    train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, criterion, device)
    val_loss, val_acc     = eval_epoch(model, val_loader, criterion, device)

    print(f"Epoch {epoch:02d} | "
          f"Train Loss: {train_loss:.4f} Acc: {train_acc:.3f} | "
          f"Val Loss: {val_loss:.4f} Acc: {val_acc:.3f}")


Epoch 01 | Train Loss: 3.4671 Acc: 0.038 | Val Loss: 3.3984 Acc: 0.053
Epoch 02 | Train Loss: 3.3933 Acc: 0.064 | Val Loss: 3.3903 Acc: 0.035
Epoch 03 | Train Loss: 3.3445 Acc: 0.075 | Val Loss: 3.3965 Acc: 0.053
Epoch 04 | Train Loss: 3.2964 Acc: 0.071 | Val Loss: 3.3795 Acc: 0.070
Epoch 05 | Train Loss: 3.2078 Acc: 0.090 | Val Loss: 3.3341 Acc: 0.105
Epoch 06 | Train Loss: 3.0656 Acc: 0.113 | Val Loss: 3.2642 Acc: 0.053
Epoch 07 | Train Loss: 2.9822 Acc: 0.117 | Val Loss: 3.2363 Acc: 0.105
Epoch 08 | Train Loss: 2.8528 Acc: 0.147 | Val Loss: 3.0798 Acc: 0.140
Epoch 09 | Train Loss: 2.7295 Acc: 0.162 | Val Loss: 3.0410 Acc: 0.140
Epoch 10 | Train Loss: 2.6315 Acc: 0.188 | Val Loss: 2.9431 Acc: 0.158


In [16]:
import json
import os

model_dir = "/content/drive/MyDrive/LSC/modelos"
os.makedirs(model_dir, exist_ok=True)

model_path = f"{model_dir}/rnn_lsc.pth"
torch.save(model.state_dict(), model_path)

labels_path = f"{model_dir}/label2idx.json"
with open(labels_path, "w") as f:
    json.dump(label2idx, f)

model_path, labels_path


('/content/drive/MyDrive/LSC/modelos/rnn_lsc.pth',
 '/content/drive/MyDrive/LSC/modelos/label2idx.json')

In [17]:
X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape


((266, 16, 150), (266,), (57, 16, 150), (57,), (57, 16, 150), (57,))

In [18]:
import tensorflow as tf
from tensorflow.keras import layers, models

num_classes = len(label2idx)
timesteps   = X_train.shape[1]   # 16
features    = X_train.shape[2]   # 150

model = models.Sequential([
    layers.Input(shape=(timesteps, features)),
    layers.Masking(mask_value=0.0),
    layers.LSTM(128, return_sequences=False),
    layers.Dropout(0.3),
    layers.Dense(64, activation="relu"),
    layers.Dropout(0.3),
    layers.Dense(num_classes, activation="softmax")
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ masking (Masking)               │ (None, 16, 150)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 128)            │       142,848 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 30)             │         1,950 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 153,054 (597.87 KB)

 Trainable params: 153,054 (597.87 KB)

 Non-trainable params: 0 (0.00 B)

In [19]:
batch_size = 16
epochs = 15  # si quieres, luego lo subes

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    batch_size=batch_size,
    epochs=epochs
)


Epoch 1/15
17/17 ━━━━━━━━━━━━━━━━━━━━ 5s 51ms/step - accuracy: 0.0498 - loss: 3.4338 - val_accuracy: 0.0351 - val_loss: 3.4133
Epoch 2/15
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.0387 - loss: 3.4152 - val_accuracy: 0.0702 - val_loss: 3.4012
Epoch 3/15
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.0502 - loss: 3.3996 - val_accuracy: 0.0175 - val_loss: 3.4060
Epoch 4/15
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.0322 - loss: 3.3793 - val_accuracy: 0.0526 - val_loss: 3.4029
Epoch 5/15
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - accuracy: 0.0316 - loss: 3.3866 - val_accuracy: 0.0877 - val_loss: 3.4028
Epoch 6/15
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.0787 - loss: 3.3572 - val_accuracy: 0.0702 - val_loss: 3.4116
Epoch 7/15
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.0550 - loss: 3.3466 - val_accuracy: 0.0351 - val_loss: 3.4322
Epoch 8/15
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.0521 - loss: 3.3809 - val_accuracy: 0.0877 - v

In [20]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print("Pérdida en test:", test_loss)
print("Accuracy en test:", test_acc)


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.0793 - loss: 3.4063
Pérdida en test: 3.3715267181396484
Accuracy en test: 0.08771929889917374


In [21]:
import os, json

# Carpeta donde guardaremos el modelo en tu Drive
save_dir = "/content/drive/MyDrive/LSC/modelos"
os.makedirs(save_dir, exist_ok=True)

model_path  = os.path.join(save_dir, "lsc_model.h5")
labels_path = os.path.join(save_dir, "labels.json")

# Guardar modelo
model.save(model_path)

# Guardar diccionario glosa -> índice
with open(labels_path, "w", encoding="utf-8") as f:
    json.dump(label2idx, f, ensure_ascii=False, indent=2)

model_path, labels_path


('/content/drive/MyDrive/LSC/modelos/lsc_model.h5',
 '/content/drive/MyDrive/LSC/modelos/labels.json')